In [ ]:
# Cell 1: Import required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import os
import glob
from PIL import Image
import shutil
import random

This cell:
- Defines our base directory and flower classes
- Counts and displays the number of images in each class
- Helps us understand our dataset distribution
- Important for identifying any class imbalance

In [ ]:
# Cell 2: Define paths and create necessary directories
base_dir = 'archive/flowers'
classes = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

# Print number of images per class
for flower_class in classes:
    class_path = os.path.join(base_dir, flower_class)
    num_images = len(os.listdir(class_path))
    print(f"Number of {flower_class} images: {num_images}")

This cell:
- Creates the directory structure for our split datasets
- Makes separate folders for train, validation, and test sets
- Creates subfolders for each flower class
- Ensures clean organization of our data

- Splits our data into training (70%), validation (15%), and test (15%) sets
- Randomly shuffles images before splitting
- Maintains class structure in each split
- Physically copies images to their respective directories

In [3]:
# Cell 3: Create train/validation/test split directories
output_base_dir = 'flowers_split'
train_dir = os.path.join(output_base_dir, 'train')
val_dir = os.path.join(output_base_dir, 'validation')
test_dir = os.path.join(output_base_dir, 'test')

# Create directories if they don't exist
for dir_path in [train_dir, val_dir, test_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        for flower_class in classes:
            os.makedirs(os.path.join(dir_path, flower_class))

# Cell 4: Split data into train/validation/test sets
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

for flower_class in classes:
    src_dir = os.path.join(base_dir, flower_class)
    images = os.listdir(src_dir)
    random.shuffle(images)
    
    n_images = len(images)
    n_train = int(n_images * train_ratio)
    n_val = int(n_images * val_ratio)
    
    # Split images
    train_images = images[:n_train]
    val_images = images[n_train:n_train + n_val]
    test_images = images[n_train + n_val:]
    
    # Copy images to respective directories
    for img in train_images:
        src = os.path.join(src_dir, img)
        dst = os.path.join(train_dir, flower_class, img)
        shutil.copy(src, dst)
        
    for img in val_images:
        src = os.path.join(src_dir, img)
        dst = os.path.join(val_dir, flower_class, img)
        shutil.copy(src, dst)
        
    for img in test_images:
        src = os.path.join(src_dir, img)
        dst = os.path.join(test_dir, flower_class, img)
        shutil.copy(src, dst)

This cell:
- Sets up image preprocessing parameters
- Standardizes image sizes to 128x128 pixels
- Sets batch size to 32 images
- Creates data generators that:
  - Rescale pixel values to [0,1]
  - Load images in batches
  - Convert class labels to categorical format

In [ ]:
# Cell 5: Set up data generators
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

class_labels = list(train_generator.class_indices.keys())

Model architecture explanation:
1. First Conv2D layer (32 filters):
   - Input layer accepting RGB images (3 channels)
   - 3x3 kernel size for feature extraction
   - ReLU activation for non-linearity

2. Three Conv2D-MaxPooling blocks:
   - Increasing filters (32 → 64 → 128)
   - MaxPooling reduces spatial dimensions
   - Helps extract hierarchical features

3. Flatten and Dense layers:
   - Converts 2D features to 1D
   - 128 neurons in dense layer
   - Dropout (0.5) for regularization
   - Final layer matches number of classes

In [ ]:
# Cell 6: Create the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(classes), activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

This cell:
- Trains the model for 15 epochs
- Uses the training generator for batched training
- Validates on validation set after each epoch
- Stores training history for later analysis

In [ ]:
# Cell 7: Train the model
EPOCHS = 15

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)


This cell:
- Creates two plots:
  1. Accuracy over time (training vs validation)
  2. Loss over time (training vs validation)
- Helps identify:
  - Model convergence
  - Overfitting
  - Training stability

In [ ]:
# Cell 8: Plot training results
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

This cell:
- Makes predictions on test set
- Creates confusion matrix
- Generates classification report with:
  - Precision
  - Recall
  - F1-score
  - Support


In [ ]:
# Cell 9: Evaluate the model and create confusion matrix
# Get predictions
test_steps = test_generator.n // test_generator.batch_size + 1
predictions = model.predict(test_generator, steps=test_steps)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes[:len(predicted_classes)]

# Create and plot confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Print classification report
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

This cell:
- Calculates ROC curves for each class
- Computes AUC (Area Under Curve)
- Visualizes model's ability to distinguish between classes
- Helps assess classification performance at various thresholds

In [ ]:
# Cell 10: ROC Curve
from sklearn.metrics import roc_curve, auc
from itertools import cycle

plt.figure(figsize=(10, 8))
n_classes = len(classes)
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(
        tf.keras.utils.to_categorical(true_classes)[:, i],
        predictions[:, i]
    )
    roc_auc[i] = auc(fpr[i], tpr[i])

colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'ROC curve of {class_labels[i]} (area = {roc_auc[i]:0.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc="lower right")
plt.show()

# Understanding the Hyperparameters:

1. Image Size (128x128):
   - Standardizes input
   - Balances detail vs computation
   - Memory efficient

2. Batch Size (32):
   - Affects training stability
   - Impacts memory usage
   - Influences gradient updates

3. Dropout Rate (0.5):
   - Prevents overfitting
   - Improves generalization
   - Standard starting value

4. Learning Rate (0.0005):
   - Controls weight update size
   - Affects training stability
   - Chosen for stable convergence